In [ ]:
import torch
from transformers import AutoTokenizer
import numpy as np
tokenizer = AutoTokenizer.from_pretrained("slauw87/bart_summarisation")

In [ ]:
sentence = "Rose has a beautiful rose garden"

input_tokens = tokenizer(sentence, padding = False, truncation = False,
                         max_length=len(sentence))
print(input_tokens)

In [ ]:
input_ids = input_tokens["input_ids"]
print(input_ids)

In [ ]:
type(input_ids)

### Simple Self attention

In [ ]:
attention_matrix = torch.zeros(len(input_ids), len(input_ids))
for row in range(len(input_ids)):
    for col in range(len(input_ids)):
        attention_matrix[row][col] = np.dot(input_ids[row], input_ids[col])

In [ ]:
attention_matrix

In [ ]:
attention_weights = torch.zeros(len(attention_matrix), len(attention_matrix))
for i, row in enumerate(attention_matrix):
    attention_weights[i] = torch.softmax(attention_matrix[i], dim =0)

In [ ]:
attention_weights

In [ ]:
out = torch.softmax(torch.tensor([0.0000e+00, 6.2200e+08, 8.4796e+05, 2.4940e+05, 6.7862e+07, 3.6363e+07,
         1.4143e+08, 4.9880e+04]), dim =0)

In [ ]:
out[3], out[4]

In [ ]:
sample = torch.tensor([0.0000e+00, 6.2200e+08, 8.4796e+05, 2.4940e+05, 6.7862e+07, 3.6363e+07,
         1.4143e+08, 4.9880e+04])
sum =0
for i in range(len(sample)):
    sum += torch.exp(sample[i])


In [ ]:
sum

In [ ]:
import torch
import numpy as np

# Your input tokens (as provided)
input_tokens = {'input_ids': [0, 24940, 34, 10, 2721, 1458, 5671, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
input_ids = torch.tensor(input_tokens['input_ids'], dtype=torch.float32)

# Create the attention matrix (dot product of input_ids with themselves)
attention_matrix = torch.matmul(input_ids.unsqueeze(1), input_ids.unsqueeze(0))

# Scale the dot product by the square root of the dimension of the token vectors (for numerical stability)
d_k = input_ids.size(-1)  # Dimension of the token vectors
scaling_factor = torch.sqrt(torch.tensor(d_k, dtype=torch.float32))
scaled_attention_matrix = attention_matrix / scaling_factor

# Apply softmax to get the attention weights
attention_weights = torch.softmax(scaled_attention_matrix, dim=-1)

print("Attention Weights:")
print(attention_weights)


In [ ]:
scaled_attention_matrix

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("slauw87/bart_summarisation")
model = AutoModel.from_pretrained("slauw87/bart_summarisation")

input_ids = tokenizer(sentence, return_tensors= "pt", padding=False, truncation= False, max_length = len(sentence))

with torch.no_grad():
    outputs = model(**input_tokens)
    embeddings = outputs.last_hidden_state

In [ ]:
embeddings[0][0:5], embeddings[0][4:8], embeddings[0]


In [ ]:
embeddings
query = embeddings
key = embeddings
value = embeddings

attention_scores = torch.dot(query, torch.t(key))

In [ ]:
query.shape

In [ ]:
torch.t(key).shape

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("slauw87/bart_summarisation")
model = AutoModel.from_pretrained("slauw87/bart_summarisation")

sentence = "Rose has a beautiful rose garden"
input_tokens = tokenizer(sentence, return_tensors="pt", padding=False, truncation=False)

# Get token embeddings from the model
with torch.no_grad():
    outputs = model(**input_tokens)
    embeddings = outputs.last_hidden_state  # Shape: (batch_size, seq_len, hidden_dim)

# Self-attention: Query, Key, Value are projections of the embeddings
query = embeddings
key = embeddings
value = embeddings

# Attention matrix: Dot product between query and key (transposed)
attention_scores = torch.matmul(query, key.transpose(-2, -1))  # Shape: (seq_len, seq_len)

# Scale by sqrt(d_k) (dimensionality of the embeddings for numerical stability)
d_k = query.size(-1)  # The hidden size
scaling_factor = torch.sqrt(torch.tensor(d_k, dtype=torch.float32))
attention_scores = attention_scores / scaling_factor

# Apply softmax to get attention weights
attention_weights = torch.softmax(attention_scores, dim=-1)

# Multiply attention weights by value to get the weighted sum (context vector)
context_vectors = torch.matmul(attention_weights, value)

print("Attention Weights:")
print(attention_weights)


In [ ]:
Rose_word = torch.tensor([4.9993e-01, 4.9993e-01, 3.7399e-05, 3.0580e-06, 1.2836e-05,
          1.7711e-05, 2.4050e-05, 4.2371e-05])

has_word = torch.tensor([3.2027e-07, 3.2027e-07, 9.9963e-01, 3.4351e-04, 1.1782e-05,
          2.8456e-06, 3.1065e-06, 5.0539e-06])
beautiful_word = torch.tensor([1.2540e-06, 1.2540e-06, 1.3442e-04, 1.3896e-02, 9.8284e-01,
          2.9472e-03, 1.5741e-04, 1.9449e-05])
rose_word = torch.tensor([1.4967e-05, 1.4967e-05, 2.8082e-04, 2.9105e-03, 2.5495e-02,
          9.6744e-01, 3.7435e-03, 1.0216e-04])
garden_word = torch.tensor([1.5248e-04, 1.5248e-04, 2.3000e-03, 2.1980e-03, 1.0216e-02,
          2.8086e-02, 9.5621e-01, 6.9020e-04])

In [ ]:
s1 = torch.dot(Rose_word, has_word)
print(f"Sim b/w Rose_word, has_word : {s1}")
s2 = torch.dot(Rose_word, beautiful_word)
print(f"Sim b/w Rose_word, beautiful_word : {s2}")
s3 = torch.dot(Rose_word, rose_word)
print(f"Sim b/w Rose_word, rose_word : {s3}")
s4 = torch.dot(Rose_word, garden_word)
print(f"Sim b/w Rose_word, garden_word : {s4}")

In [ ]:
s1 = torch.dot(rose_word, has_word)
print(f"Sim b/w rose_word, has_word : {s1}")
s2 = torch.dot(rose_word, beautiful_word)
print(f"Sim b/w rose_word, beautiful_word : {s2}")
s3 = torch.dot(rose_word, rose_word)
print(f"Sim b/w rose_word, rose_word : {s3}")
s4 = torch.dot(rose_word, garden_word)
print(f"Sim b/w rose_word, garden_word : {s4}")

In [ ]:
import gensim.downloader as api
from nltk.tokenize import word_tokenize

# Load pre-trained Word2Vec model
word2vec_model = api.load("word2vec-google-news-300")

# Sentence
sentence = "The car made a quick left turn and sped along the road."

# Tokenize the sentence
tokens = word_tokenize(sentence.lower())

# Get word embeddings for each word in the sentence
word_embeddings = [word2vec_model[word] for word in tokens if word in word2vec_model]

# Combine embeddings (e.g., average)
import numpy as np
sentence_embedding = np.mean(word_embeddings, axis=0)


In [ ]:
sentence_embedding

In [ ]:
from importlib.metadata import version
import tiktoken

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
text = "rose has a beutiful rose garden"
import torch.nn.functional as F
## Lets assign the words with word embeddings
inputs = torch.tensor(
    [[0.43, 0.15, 0.89],
     [0.55, 0.87, 0.66],
     [0.57, 0.85, 0.64],
     [0.22,0.58, 0.33],
     [0.43, 0.15, 0.89],
     [0.05, 0.80, 0.55]]
)

In [ ]:
inputs.shape

In [ ]:
d_in = inputs.shape[1]
d_out = 2

In [ ]:
### Query, Key, values weight matrices
W_q = torch.rand(d_in, d_out, dtype = torch.float32, requires_grad=False)
W_k = torch.rand(d_in, d_out, dtype = torch.float32, requires_grad=False)
W_v = torch.rand(d_in, d_out, dtype = torch.float32, requires_grad=False)

In [ ]:
## Calculating the attention scores
Q = inputs@W_q
K = inputs@W_k
V = inputs@W_v
attn_scores = Q@K.t()

In [ ]:
attn_scores

# [[0.8089, 1.6977, 1.6636, 1.0353, 0.8089, 1.4218],<----- rose
# [1.1166, 2.3618, 2.3143, 1.4418, 1.1166, 1.9798],
# [1.1020, 2.3304, 2.2836, 1.4226, 1.1020, 1.9535],
# [0.6146, 1.3032, 1.2770, 0.7958, 0.6146, 1.0928],
# [0.8089, 1.6977, 1.6636, 1.0353, 0.8089, 1.4218],<----- rose
# [0.7968, 1.6927, 1.6587, 1.0339, 0.7968, 1.4196]]


In [ ]:
attention_matrix = attn_scores/d_in**0.5
attention_matrix  = F.softmax(attention_matrix, dim=-1)

In [ ]:
context_vectors = attention_matrix@V
context_vectors

In [ ]:
import torch
import torch.nn.functional as F
import math

# Function to create positional encoding
def get_positional_encoding(seq_len, d_model):
    positional_encoding = torch.zeros(seq_len, d_model)
    for pos in range(seq_len):
        print(pos)
        for i in range(0, d_model, 2):
            positional_encoding[pos, i] = math.sin(pos / (10000 ** (i / d_model)))
            print("sin")
            if i + 1 < d_model:
                positional_encoding[pos, i + 1] = math.cos(pos / (10000 ** (i / d_model)))
                print("cos")
    return positional_encoding

# Sentence: "rose has a beautiful rose garden"
# Using the same initial embedding for both occurrences of "rose"
inputs = torch.tensor(
    [
        [0.56, 0.72, 0.89],  # rose (first occurrence)
        [0.45, 0.65, 0.55],  # has
        [0.31, 0.77, 0.22],  # a
        [0.12, 0.85, 0.33],  # beautiful
        [0.56, 0.72, 0.89],  # rose (second occurrence) - same embedding
        [0.05, 0.80, 0.55]   # garden
    ]
)

# Dimension of the input embeddings (d_model)
d_model = inputs.shape[1]

# Create positional encoding
seq_len = inputs.size(0)
positional_encoding = get_positional_encoding(seq_len, d_model)

# Add positional encoding to the input embeddings
inputs += positional_encoding

# Initialize weight matrices for query, key, and value transformations
d_k = d_model  # Dimension for query and key (same as input embeddings)
d_v = 2        # Output size for value embeddings (reduced dimension for simplicity)

# Random weight matrices for Queries, Keys, and Values
W_q = torch.rand(d_model, d_k)  # Query weight matrix
W_k = torch.rand(d_model, d_k)  # Key weight matrix
W_v = torch.rand(d_model, d_v)  # Value weight matrix

# Step 1: Calculate Queries (Q), Keys (K), and Values (V)
Q = inputs @ W_q  # Queries: shape (6, 3) @ (3, 3) = (6, 3)
K = inputs @ W_k  # Keys: shape (6, 3) @ (3, 3) = (6, 3)
V = inputs @ W_v  # Values: shape (6, 3) @ (3, 2) = (6, 2)

# Step 2: Calculate attention scores (QK^T / sqrt(d_k))
attn_scores = Q @ K.T  # shape (6, 3) @ (3, 6) = (6, 6)
attn_scores /= d_k ** 0.5  # Scale by sqrt(d_k)

# Step 3: Apply softmax to get attention weights
attn_weights = F.softmax(attn_scores, dim=-1)  # shape (6, 6)

# Step 4: Get context vectors by multiplying attention weights with Values (V)
context_vectors = attn_weights @ V  # shape (6, 6) @ (6, 2) = (6, 2)

# Print the attention weights and final context vectors
print("Attention weights:\n", attn_weights)
print("Context vectors (after self-attention):\n", context_vectors)


In [ ]:
inputs = torch.tensor(
    [
        [],  # rose (first occurrence)
        [],  # has
        [0.31, 0.77, 0.22],  # a
        [0.12, 0.85, 0.33],  # beautiful
        [0.56, 0.72, 0.89],  # rose (second occurrence) - same embedding
        [0.05, 0.80, 0.55]   # garden
    ]
)


In [20]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89],
     [0.55, 0.87, 0.66],
     [0.57, 0.85, 0.64],
     [0.22,0.58, 0.33],
     [0.77, 0.25, 0.10],
     [0.05, 0.80, 0.55]]
)

In [23]:
d_in = inputs.shape[1]
d_out = inputs.shape[1]
torch.manual_seed(13)
W_q = torch.rand(d_in, d_out, dtype = torch.float32)
W_k = torch.rand(d_in, d_out, dtype = torch.float32)
W_v = torch.rand(d_in, d_out, dtype = torch.float32)

Q = inputs@W_q # Query vector
K = inputs@W_k # Key vector
V = inputs@W_v # Value vector

### Calculate attention scores

attn_scores = torch.softmax((Q@K.t())/(inputs.shape[0]**0.5), dim =1)

context_vectors = attn_scores@V

In [24]:
attn_scores

tensor([[0.1431, 0.2080, 0.2055, 0.1450, 0.1305, 0.1678],
        [0.1423, 0.2112, 0.2087, 0.1425, 0.1296, 0.1657],
        [0.1426, 0.2107, 0.2082, 0.1428, 0.1300, 0.1657],
        [0.1546, 0.1889, 0.1877, 0.1545, 0.1476, 0.1666],
        [0.1541, 0.1898, 0.1886, 0.1540, 0.1466, 0.1668],
        [0.1512, 0.1950, 0.1935, 0.1513, 0.1425, 0.1665]])

In [ ]:
Q@K.t()/inputs.shape[0]**0.5

In [10]:
### Self attention class
import torch.nn as nn
import torch
class SelfAttention_V1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()

        self.W_q = nn.Parameter(torch.rand(d_in, d_out, dtype = torch.float32))
        self.W_k = nn.Parameter(torch.rand(d_in, d_out, dtype = torch.float32))
        self.W_v = nn.Parameter(torch.rand(d_in, d_out, dtype = torch.float32))

    def forward(self, x):
        keys = x@self.W_k
        query = x@self.W_q
        values = x@self.W_v

        attn_scores = torch.softmax((query@keys.T)/(keys.shape[1]**0.5), dim =1)

        context_vectors = attn_scores@values

        return context_vectors
        

In [11]:
torch.manual_seed(42)
inputs = torch.tensor(
    [[0.43, 0.15, 0.89],
     [0.55, 0.87, 0.66],
     [0.57, 0.85, 0.64],
     [0.22,0.58, 0.33],
     [0.77, 0.25, 0.10],
     [0.05, 0.80, 0.55]]
)
d_in = inputs.shape[1]
d_out = inputs.shape[1]
sa_v1 = SelfAttention_V1(d_in, 2)
print(sa_v1(inputs))

tensor([[1.3751, 0.8610],
        [1.4201, 0.8892],
        [1.4198, 0.8890],
        [1.3533, 0.8476],
        [1.3746, 0.8606],
        [1.3620, 0.8532]], grad_fn=<MmBackward0>)
